In [1]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from xfem import *
from xfem.mlset import *
from math import pi,e
from numpy import linspace


importing ngsxfem-2.1.2504


In [2]:
aperture = 0.1 # width
H = 0.6 # height
outer = Rectangle(1, 1).Face()
outer.edges.name="outer"
# outer.edges.Max(X).name = "r"
# outer.edges.Min(X).name = "l"
# outer.edges.Min(Y).name = "b"
# outer.edges.Max(Y).name = "t"

Omegaf = MoveTo(0.5-aperture/2, 0.5 - H/2).Rectangle(aperture, H).Face()
Omegaf.edges.name="interface"
Omegap = outer - Omegaf

Omegaf.faces.name="Omegaf"
Omegaf.faces.col = (1, 0, 0)
Omegap.faces.name="Omegap"

geo = Glue([Omegaf, Omegap])
# Draw(geo);

In [3]:
mh = 1/128
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=mh))
Draw(mesh);
# print(mesh.GetBoundaries())
# print(mesh.GetMaterials())

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [4]:
# for v in mesh.vertices:
#     print (v, v.point)

# for el in mesh.Elements(VOL):
#     print(type(el))
#     print ("vertices: ", el.vertices)   # get vertices of an element
#     print ("edges: ", el.edges)   

In [5]:
def GetInterfaceFaces(mesh):
    fes = FacetFESpace(mesh, order=0)
    g = GridFunction(fes)
    
    # domain number function
    dom = CoefficientFunction([1 if m=="Omegap" else 2 for m in mesh.GetMaterials()])
    
    a = BilinearForm(fes, symmetric=True)
    a += fes.TrialFunction()*fes.TestFunction() * dx(element_boundary=True)
    a.Assemble()
    
    f = LinearForm(fes)
    f += dom * fes.TestFunction() * dx(element_boundary=True)
    f.Assemble()
    
    g.vec.data = a.mat.Inverse() * f.vec
    
    interface_facets = []
    for i, val in enumerate(g.vec):
        if abs(val - 1.5) < 0.25:   # 1.5 = (1+2)/2
            interface_facets.append(i)
    
    # print("Interface facets:", interface_facets)
    interface_marker = BitArray(fes.ndof)
    interface_marker[:] = False
    
    for f in interface_facets:
        interface_marker[f] = True
    return interface_marker
interFaces = GetInterfaceFaces(mesh)
# ba_surround_facets = GetElementsWithNeighborFacets(mesh,interFaces)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets")  

In [6]:
# penalty parameters
order_eta = 2
order_pp = 1
order_pf = 1

beta_eta = 100
beta_p = 10
sigma = 20
# physical parameters
mu  = 10
lam = 100
alpha = 1
Kp = 1
Kf = 1

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [7]:
# 定义解析解
# eta_x = CF(1)
# eta_y = CF(2)
eta_x = 1 - 2*y
eta_y = 1 + 2*x
exact_eta = CF((eta_x,eta_y))
# exact_p = 1e4*(x-0.3)**2*(x-0.7)**2*(y-0.2)**2*(y-0.8)**2
exact_p = sin(pi*x)*sin(pi*y)

# strain tensor
epsilon_xx = eta_x.Diff(x)
epsilon_yy = eta_y.Diff(y) 
epsilon_xy = 0.5*(eta_x.Diff(y) +  eta_y.Diff(x))

# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_xx - alpha*exact_p
sigma_yy = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_yy - alpha*exact_p
sigma_xy = 2*mu*epsilon_xy

# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y))

# 向量形式
fe = CF( (f_x,f_y) )
fp = -Kp*(exact_p.Diff(x).Diff(x)+exact_p.Diff(y).Diff(y))
ff = -Kf*(exact_p.Diff(x).Diff(x)+exact_p.Diff(y).Diff(y))

etaD = exact_eta
pD = exact_p
# eta_gamma = 

In [8]:
# DG spaces
Eh = VectorL2(mesh, definedon="Omegap", order=order_eta, dirichlet=".*", dgjumps=True) # space for velocity
P = L2(mesh, order = order_eta-1, dirichlet=".*", dgjumps=True) # space for pressure
E = Compress(Eh)
fes = E*P
(eta,p), (xi,q) = fes.TnT()

# define integral operators
dOmegap = dx(definedon=mesh.Materials("Omegap")) # integral on Omegap
dOmegaf = dx(definedon=mesh.Materials("Omegaf")) # integral on Omegaf
dso = ds(skeleton=True, definedon=mesh.Boundaries("outer")) # integral on outer faces

# dgamma = dx(skeleton=True, definedon=mesh.Boundaries("interface")) # integral on interfaces
dgamma = dx(skeleton=True, definedonelements=interFaces)
dsip = dx(skeleton=True, definedon=mesh.Materials("Omegap")) # integral on interior faces of Omegap
dsif = dx(skeleton=True, definedon=mesh.Materials("Omegaf")) # integral on interior faces of Omegaf


In [9]:
##### test for dsif/dsip integral operator
# p,q = F.TnT()
# ah = BilinearForm(F)
# jump_p = p - p.Other()
# jump_q = q - q.Other()
# # ah += p*q*dOmegaf
# ah += jump_p * jump_q * dsif
# ah.Assemble()
# print(ah.mat)

In [10]:
# Define the jumps and the averages
jump_eta = eta - eta.Other()
jump_xi = xi - xi.Other()
jump_p = p - p.Other()
jump_q = q - q.Other()
h = specialcf.mesh_size  
n = specialcf.normal(2)
strain_eta = Sym(Grad(eta))
strain_xi = Sym(Grad(xi))
mean_stress_eta = 0.5*(Stress(Sym(Grad(eta)))+Stress(Sym(Grad(eta.Other()))))*n
mean_stress_xi = 0.5*(Stress(Sym(Grad(xi)))+Stress(Sym(Grad(xi.Other()))))*n

mean_dpdn = 0.5*Kp*(grad(p)+grad(p.Other()))*n
mean_dqdn = 0.5*Kp*(grad(q)+grad(q.Other()))*n

mean_p = 0.5*(p + p.Other())
mean_q = 0.5*(q + q.Other())
 

In [11]:
# p,q = E.TnT()
# mh = BilinearForm(E)
# jump_p = p - p.Other()
# jump_q = q - q.Other()
# mh += jump_p*jump_q*dgamma
# mh.Assemble()
# print(mh.mat)

In [12]:
ah = BilinearForm(fes)
# Ae
ah += 2*mu*InnerProduct(strain_eta,strain_xi)*dOmegap + lam*div(eta)*div(xi)*dOmegap \
        - (InnerProduct(mean_stress_eta,jump_xi) + InnerProduct(mean_stress_xi,jump_eta) - beta_eta/h*InnerProduct(jump_eta,jump_xi))*dsip \
        - (InnerProduct(Stress(Sym(Grad(eta)))*n,xi) + InnerProduct(Stress(Sym(Grad(xi)))*n,eta) - beta_eta/h*InnerProduct(eta,xi))*dso
# Be
ah += -alpha*(div(xi)*p*dOmegap - mean_p*jump_xi*n*dsip - p*xi*n*dso)
# I
ah += p*n*xi*dgamma

# Ap
ah += Kp*grad(p)*grad(q)*dOmegap \
        - (mean_dpdn*jump_q + mean_dqdn*jump_p - beta_p/h*jump_p*jump_q)*dsip \
        - (Kp*grad(p)*n*q + Kp*grad(q)*n*p - beta_p/h*p*q)*dso

# Af
ah += Kf*grad(p)*grad(q)*dOmegaf \
        - (mean_dpdn*jump_q + mean_dqdn*jump_p - beta_p/h*jump_p*jump_q)*dsif 

ah += - 0.5*(Kp*grad(p) + Kf*grad(p.Other()))*n * jump_q * dgamma
ah += - 0.5*(Kp*grad(q) + Kf*grad(q.Other()))*n * jump_p * dgamma # 对称项

ah += sigma/h * jump_p*jump_q * dgamma # continuity for pressure

ah.Assemble()

In [13]:
# r.h.s
fh = LinearForm(fes)
fh += fe*xi*dOmegap - InnerProduct(etaD,Stress(Sym(Grad(xi)))*n)*dso + beta_eta/h*etaD*xi*dso
fh += fp*q*dOmegap - Kp*grad(q)*n*pD*dso + beta_p/h*pD*q*dso
fh += ff*q*dOmegaf

fh.Assemble()

In [14]:
gfu = GridFunction(fes)
gfu.vec.data = ah.mat.Inverse() * fh.vec

In [15]:
error_eta = sqrt(Integrate((gfu.components[0] - exact_eta)**2*dOmegap, mesh))
print(error_eta)
error_pp = sqrt(Integrate((gfu.components[1] - exact_p)**2*dx, mesh))
print(error_pp)

0.0017629645381287385
3.5025493878063985e-05


In [19]:
Draw(gfu.components[0],mesh)
exact_etah = GridFunction(E)
exact_etah.Set(exact_eta)
Draw(exact_etah-gfu.components[0],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [23]:
# Draw(gfu.components[1],mesh)
Draw(exact_p-gfu.components[1],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene